In [1]:
print('Assignment 0')

Assignment 0


In [1]:
import cv2
import os
import shutil
import numpy as np

# Video -> Frames

In [2]:
# Function to create a folder
def createFolder(name):
    try:
        os.mkdir(name)
        print('Folder created!')
    except:
        print('Folder already exists!')

# Create opFolder
def createOpFolder(vid, opFolder):
    # If the video file is not found or some other error occured in opening the file
    if not vid.isOpened():
        print('Error while opening the file')
    # If the file is opened, create an output folder to save frames of the video
    else:
        # If ouput folder exists, delete its data
        if os.path.isdir(opFolder):
            try:
                # Delete existing output folder and create a new one
                shutil.rmtree(opFolder)
                print('Old folder deleted')
                createFolder(opFolder)
            except:
                print('Old folder not deleted')
        else:
            createFolder(opFolder)
        
def convertVidToIm(filename, opFolder):
    vid = cv2.VideoCapture(filename)

    createOpFolder(vid, opFolder)

    # Frames array
    opFrames = []
    frameCount = 0
    while(vid.isOpened()):
        # Return is True if the read() returns a frame
        # frame is the frame of the video
        ret, frame = vid.read()

        # If ret is True, show the frame
        if ret:
            frameCount = frameCount + 1
            opFrames.append(frame)        
            cv2.imshow('frame',frame)

            # Save frame in output folder
            fn = opFolder + '/%d.jpg'%frameCount
            cv2.imwrite(fn, frame)

            # Press 'q' to quit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break


    opFrames = np.asarray(opFrames)
    
    vid.release()
    cv2.destroyAllWindows()
    return opFrames

In [5]:
# Constants
opFolder = 'output'
# videoFile = 'input/V3.mp4'
videoFile = 'bgVid.mp4'

frames = convertVidToIm(videoFile, opFolder)

Folder created!


# Images -> Video

In [3]:
# Reads image in sorted order
def getImages(inputFolder):
    fileType = ''
    imFiles = []

    for a,b,files in os.walk(inputFolder):
        for file in sorted(files):
            name = file.split('.')
            imFiles.append(name[0])

            if not fileType:
                fileType = name[1]
                frame = cv2.imread(inputFolder+'/'+file)
                frameSize = (frame.shape[1],frame.shape[0])
 
    ims = np.asarray(list(cv2.imread(inputFolder + '/%s'%im + '.' + fileType) for im in sorted(imFiles, key=int)))
    
    return ims, frameSize

# Converts a list of images into a single video
def convertImToVid(images, fps, frameSize):
    name = 'outputVideo.mp4'
    opvid = cv2.VideoWriter(name, cv2.VideoWriter_fourcc('M','J','P','G'), fps, frameSize, True)
    for im in images:
        opvid.write(im)
#         cv2.imshow('frame',im)
#         if cv2.waitKey(20) & 0xFF == ord('q'):
#             break
    
    opvid.release()
    cv2.destroyAllWindows()
    
    playVideo(name)

In [16]:
ipFolder = 'output'
opvideoFile = 'outputVideo.avi'

# Read images from the directory
ims, fs = getImages(ipFolder)
convertImToVid(ims, 0.9, fs)

# Capture webcam video

In [25]:
# Constants
webopFolder = 'weboutput'

# Capture and show webcam video and save as images
convertVidToIm(0, webopFolder)

opvideoFile = 'outputVideo.avi'

# Read images from the directory
ims, fs = getImages(webopFolder)
convertImToVid(ims, 20, fs)

Old folder deleted
Folder created!


# Chroma Keying

In [4]:
def playVideo(filename):
    vid = cv2.VideoCapture(filename)

    while(vid.isOpened()):
        # Return is True if the read() returns a frame
        # frame is the frame of the video
        ret, frame = vid.read()

        # If ret is True, add the frame
        if ret:
            cv2.imshow('frame',frame)

            # Press 'q' to quit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break
 
    vid.release()
    cv2.destroyAllWindows()

# fgVid = Foreground video
# bgVid = Background video
def chromaKeying(fgVid, bgVid):
    fgFolder = 'fg'
    bgFolder = 'bg'
    fgFrames = convertVidToIm(fgVid, fgFolder)
    bgFrames = convertVidToIm(bgVid, bgFolder)
    
    if not ((len(fgFrames) == 0) & (len(bgFrames) == 0)):
        print(len(fgFrames))
        print(len(bgFrames))
        
        A = 1.0
        B = 1.0    
        newFgFrames = []

        fs = fgFrames[0].shape

        for fg, bg in zip(fgFrames, bgFrames):

            # For each pixel, check whether fg or bg
            for i in range(0, fs[0]):
                for j in range(0,fs[1]):
                    fgVal = fg[i,j]
                    alpha = np.int16(fgVal[0]) + np.int16(fgVal[2]) - np.int16(fgVal[1])
                    # Replace with background pixel value if alpha is less than 0
                    if alpha <= 0:
                        fg[i,j] = bg[i,j]

                    # Remove green spill
                    fg[i,j] = [fgVal[0], min(fgVal[1], fgVal[2]), fgVal[2]]

            newFgFrames.append(fg)

        convertImToVid(newFgFrames, 25, (fs[1], fs[0]))
    

In [26]:
# Constants
fg = 'fgVid.mp4'
bg = 'bgVid.mp4'

chromaKeying(fg, bg)

Old folder deleted
Folder created!
Old folder deleted
Folder created!
393
99


# Chroma Keying using HSV model

In [74]:
import cv2
import numpy as np

fgFile = 'fgVid.mp4'
bgFile = 'bgVid.mp4'
fgOp = 'fg'
bgOp = 'bg'
keyMickey = 0.3491
lwrB = 0.340
uprB = 0.355

fgFrames = convertVidToIm(fgFile, fgOp)
bgFrames = convertVidToIm(bgFile, bgOp)

if not ((len(fgFrames) == 0) & (len(bgFrames) == 0)):
        print(len(fgFrames))
        print(len(bgFrames))
        
        A = 1.0
        B = 1.0    
        newFgFrames = []

        fs = fgFrames[0].shape

        for fgF, bgF in zip(fgFrames, bgFrames):
            hsv = cv2.cvtColor(fgF, cv2.COLOR_BGR2HSV)
            
            hsv[np.where((hsv[0] >= lwrB & hsv[0] <= uprB))] = bgF[np.where((hsv[0] >= lwrB & hsv[0] <= uprB))]

Old folder deleted
Folder created!
Old folder deleted
Folder created!
398
894


TypeError: ufunc 'bitwise_and' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [35]:
# fg = cv2.imread('44.jpg')
# bg = cv2.imread('12.jpg')
lhb = 45
uhb = 65
lsb = 150
usb = 255

# fgVid = Foreground video
# bgVid = Background video
def chromaKeyinghsv(fgVid, bgVid):
    fgFolder = 'fg'
    bgFolder = 'bg'
    fgFrames = convertVidToIm(fgVid, fgFolder)
    bgFrames = convertVidToIm(bgVid, bgFolder)
    
    if not ((len(fgFrames) == 0) & (len(bgFrames) == 0)):  
        newFgFrames = []
        fs = fgFrames[0].shape

        for fg, bg in zip(fgFrames, bgFrames):
            hsv = cv2.cvtColor(fg, cv2.COLOR_BGR2HSV)
            hc = np.logical_and(hsv[:,:,0] >= lhb, hsv[:,:,0] <= uhb)
            sc = np.logical_and(hsv[:,:,1] >= lsb, hsv[:,:,1] <= usb)
            fg[np.where(np.logical_and(hc,sc))]= bg[np.where(np.logical_and(hc, sc))]
            fg[:,:,1] = np.min([fg[:,:,1],fg[:,:,2]], axis = 0)

            newFgFrames.append(fg)

        convertImToVid(newFgFrames, 25, (fs[1], fs[0]))

# Constants
fg = 'fgVid.mp4'
bg = 'bgVid.mp4'

chromaKeyinghsv(fg, bg)
        
# hsv = cv2.cvtColor(fg, cv2.COLOR_BGR2HSV)
# fg[np.where(np.logical_and(hsv[:,:,0] >= lwrB, hsv[:,:,0] <= uprB))] = bg[np.where(np.logical_and(hsv[:,:,0] >= lwrB, hsv[:,:,0] <= uprB))]

# fg[:,:,1] = np.min([fg[:,:,1],fg[:,:,2]], axis = 0)

# cv2.imwrite('finalImage.jpg', fg)
# cv2.imwrite('finalImagehsv.jpg', hsv)

Old folder deleted
Folder created!
Old folder deleted
Folder created!


In [9]:
fg = cv2.imread('1.jpg')
print(fg[0,0,:])
print(fg[30,30,:])
fhsv = cv2.cvtColor(fg, cv2.COLOR_BGR2HSV)
print(fhsv[0,0,:])

[  9 180  48]
[ 11 190  57]
[ 53 242 180]


In [51]:
            hsv = cv2.cvtColor(fg, cv2.COLOR_BGR2HSV)

            hc = np.logical_and(hsv[:,:,0] >= lhb, hsv[:,:,0] <= uhb)
            sc = np.logical_and(hsv[:,:,1] >= lsb, hsv[:,:,1] <= usb)
            fg[np.where(np.logical_and(hc,sc))]= bg[np.where(np.logical_and(hc, sc))]
            fg[:,:,1] = np.min([fg[:,:,1],fg[:,:,2]], axis = 0)


array([[199, 199, 199, ..., 196, 197, 198],
       [199, 199, 199, ..., 198, 199, 200],
       [199, 199, 199, ..., 200, 202, 203],
       ...,
       [199, 199, 199, ..., 198, 198, 198],
       [199, 199, 199, ..., 198, 198, 198],
       [199, 199, 199, ..., 198, 198, 198]], dtype=uint8)